In [23]:
# Libraries to be used
import re
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from requests.exceptions import Timeout
from IPython.display import clear_output

In [24]:
# sending a request to the website
html = requests.get("https://www.ccarprice.com/ke/") 

# Checking the status of the website
if html.status_code == 200:
    print("Connection established successfully!")
else:
    print(f"Connection failed with status code {html.status_code}")
    

Connection established successfully!


In [25]:
# converting the html content to a soup to scrap data with
soup = BeautifulSoup(html.content, 'html.parser')

In [26]:
class Car_Brand_Scraper:
    def __init__(self, soup):
        self.soup = soup

    def extract_brand_links(self):
        """Extract all brand links from the page."""
        try:
            divs_1 = self.soup.find("div", class_="vertical-menu") \
                              .find("label").find("div", class_="show1") \
                              .find_all("a", class_="brnd")

            brand_links = [x.get("href") for x in divs_1 if x.get("href")]
            return brand_links

        except AttributeError:
            print("Error: The website structure may have changed or the expected elements were not found.")
            return []

    def display_links(self):
        """Display the links."""
        brand_links = self.extract_brand_links()
        if brand_links:
            print(f'The website "https://www.ccarprice.com/au/" has data of {len(brand_links)} car brands.')
            print('\nAll Brands Links:')
            for idx, link in enumerate(brand_links, 1):
                print(f'{idx}. {link}')
        else:
            print("No brand links found.")

# Instantiating the Car Brand Scrapper Class
scraper = Car_Brand_Scraper(soup)

# Assigning the brand links to brand_links variable
brand_links = scraper.extract_brand_links()

#  Calling the display_links class
scraper.display_links()


The website "https://www.ccarprice.com/au/" has data of 88 car brands.

All Brands Links:
1. https://www.ccarprice.com/ke/honda-car-prices-in-Kenya-1
2. https://www.ccarprice.com/ke/bmw-car-prices-in-Kenya-2
3. https://www.ccarprice.com/ke/lexus-car-prices-in-Kenya-3
4. https://www.ccarprice.com/ke/toyota-car-prices-in-Kenya-4
5. https://www.ccarprice.com/ke/nissan-car-prices-in-Kenya-5
6. https://www.ccarprice.com/ke/audi-car-prices-in-Kenya-6
7. https://www.ccarprice.com/ke/kia-car-prices-in-Kenya-7
8. https://www.ccarprice.com/ke/mitsubishi-car-prices-in-Kenya-8
9. https://www.ccarprice.com/ke/hyundai-car-prices-in-Kenya-9
10. https://www.ccarprice.com/ke/chevrolet-car-prices-in-Kenya-10
11. https://www.ccarprice.com/ke/ford-car-prices-in-Kenya-102
12. https://www.ccarprice.com/ke/mercedes-car-prices-in-Kenya-103
13. https://www.ccarprice.com/ke/porsche-car-prices-in-Kenya-104
14. https://www.ccarprice.com/ke/suzuki-car-prices-in-Kenya-105
15. https://www.ccarprice.com/ke/infiniti-c

In [27]:
# empty dictionary to store brand and link pairs  
car_links = {}

def format_brand_name(brand_link):
    """Retrieving the brand using regex"""
    # Using the regular expression to extract the brand name
    brand = re.search(r'https:\/\/www\.ccarprice\.com\/ke\/(.+)\-car', str(brand_link)).group(1)

    # Split by hyphen, capitalize each word, and join back
    return ' '.join(word.capitalize() for word in brand.split('-'))

def fetch_car_links_by_brand(brand_link, formatted_brand_name):
    """Fetch all car links for a specific brand from the given brand link."""
    # Request the HTML content
    html1 = requests.get(brand_link)
    soup1 = BeautifulSoup(html1.content, 'html.parser')

    # Find the relevant div containing all the car listings
    cars_by_brand = soup1.body.find("div", {"id": "page"}).find_all("div", {"id": "pbox", "class": "price-cover"})[-1].find_all("div", {"id": "pbox", "class": "listing"})

    # Initialize a list to hold all car links
    link_of_cars_by_brand = []

    # Iterate over each car listing and extract the href
    for car in cars_by_brand:
        # Check if the car has a link and if it is not labeled as "Coming soon"
        if car.a:
            text = car.getText().strip()
            if "Coming soon" not in text:
                car_link = car.a.get("href")
                link_of_cars_by_brand.append(car_link)

    # Return the result as a dictionary
    return {formatted_brand_name: link_of_cars_by_brand}

In [28]:
# Empty dictionary to store the car brand and car link pairs 
car_links = {}

for n, each_brand_link in enumerate(brand_links):
    #Retrieving the brand name from the link
    formatted_brand_name = format_brand_name(each_brand_link)

    # Retrieving all car links associated to that brand
    brand_car_links = fetch_car_links_by_brand(each_brand_link, formatted_brand_name)

    # updating the empty dictionary
    car_links.update(brand_car_links)

# printing the results
car_links

{'Honda': ['https://www.ccarprice.com/ke/honda-cr-v-ex-awd-2025-price-in-kenya-23534',
  'https://www.ccarprice.com/ke/honda-cr-v-ex-2wd-2025-price-in-kenya-23532',
  'https://www.ccarprice.com/ke/honda-elevate-signature-black-edition-2025-price-in-kenya-23518',
  'https://www.ccarprice.com/ke/honda-elevate-black-edition-2025-price-in-kenya-23517',
  'https://www.ccarprice.com/ke/honda-pilot-trailsport-2026-price-in-kenya-23513',
  'https://www.ccarprice.com/ke/honda-cr-v-hybrid-sport-touring-awd-2025-price-in-kenya-23509',
  'https://www.ccarprice.com/ke/honda-civic-sedan-hybrid-sport-cvt-2025-price-in-kenya-23492',
  'https://www.ccarprice.com/ke/honda-cr-v-lx-awd-2025-price-in-kenya-23483',
  'https://www.ccarprice.com/ke/honda-cr-v-lx-2wd-2025-price-in-kenya-23482',
  'https://www.ccarprice.com/ke/honda-accord-sport-l-sedan-2025-price-in-kenya-23459',
  'https://www.ccarprice.com/ke/honda-accord-touring-sedan-2025-price-in-kenya-23444',
  'https://www.ccarprice.com/ke/honda-civic-s

In [29]:
print('The website "https://www.ccarprice.com/ke/" has the following data:\n')

car_data = {
    "Brand": list(car_links.keys()),
    "Number of Cars": [len(cars) for cars in car_links.values()]
}

# Create the DataFrame
car_df = pd.DataFrame(car_data)

# Add a total row
total_cars_details = car_df["Number of Cars"].sum()
car_df.loc[len(car_df.index)] = ["Total", total_cars_details]

# viewing the first 5 entries
car_df

The website "https://www.ccarprice.com/ke/" has the following data:



,Brand,Number of Cars
0,Honda,60
1,Bmw,60
2,Lexus,60
3,Toyota,60
4,Nissan,60
...,...,...
84,Datsun,20
85,Lucid,33
86,Scout,3
87,Xiaomi,6


In [30]:
car_links

{'Honda': ['https://www.ccarprice.com/ke/honda-cr-v-ex-awd-2025-price-in-kenya-23534',
  'https://www.ccarprice.com/ke/honda-cr-v-ex-2wd-2025-price-in-kenya-23532',
  'https://www.ccarprice.com/ke/honda-elevate-signature-black-edition-2025-price-in-kenya-23518',
  'https://www.ccarprice.com/ke/honda-elevate-black-edition-2025-price-in-kenya-23517',
  'https://www.ccarprice.com/ke/honda-pilot-trailsport-2026-price-in-kenya-23513',
  'https://www.ccarprice.com/ke/honda-cr-v-hybrid-sport-touring-awd-2025-price-in-kenya-23509',
  'https://www.ccarprice.com/ke/honda-civic-sedan-hybrid-sport-cvt-2025-price-in-kenya-23492',
  'https://www.ccarprice.com/ke/honda-cr-v-lx-awd-2025-price-in-kenya-23483',
  'https://www.ccarprice.com/ke/honda-cr-v-lx-2wd-2025-price-in-kenya-23482',
  'https://www.ccarprice.com/ke/honda-accord-sport-l-sedan-2025-price-in-kenya-23459',
  'https://www.ccarprice.com/ke/honda-accord-touring-sedan-2025-price-in-kenya-23444',
  'https://www.ccarprice.com/ke/honda-civic-s

In [36]:
# 
all_car_details = pd.DataFrame()
index_number = 0
count = 0
for n, (each_brand, list_of_cars_links) in enumerate(car_links.items()):
   for m, link_of_each_car in enumerate(list_of_cars_links):
        car_details_grouped_by_brand = {}
        
        try:
            html2 = requests.get(link_of_each_car, timeout=10) 
        except:
            continue 
        else: 
            soup2 = BeautifulSoup(html2.content, 'html.parser')

            price_details = soup2.select_one('div.detail-price#pbox')
            if price_details:
                price_text = price_details.get_text(strip=True)
            
                # Extract prices using regex
                kes_match = re.search(r'Price in KES:\s*([\d,]+)', price_text)
                price_in_kes = kes_match.group(1).replace(',', '') if kes_match else 'N/A'
            
                usd_match = re.search(r'Price in USD:\s*\$([\d,]+)', price_text)
                price_in_usd = usd_match.group(1).replace(',', '') if usd_match else 'N/A'

            else:
                print("Price details not found.")              

            # Extract year safely
            year_UNFILTERED = soup2.select_one('div#pbox.detail-cover div.tr div.td2')
            
            if year_UNFILTERED:
                year = year_UNFILTERED.text.strip()
                try:
                    year = re.search('.*(\d{4}).*', year).group(1)
                except:
                    year = np.nan
            else:
                year = np.nan

            try:
                image_tag = soup2.find("img", {"itemprop": "image"})
                image_url = image_tag["src"] if image_tag else np.nan
            except:
                image_url = np.nan

            car_details_grouped_by_brand['brand'] = each_brand
            car_details_grouped_by_brand['year'] = year
            car_details_grouped_by_brand['price_in_kes'] = price_in_kes
            car_details_grouped_by_brand['price_in_usd'] = price_in_usd
            car_details_grouped_by_brand["image_url"]= image_url
            car_details_grouped_by_brand['car_link']=link_of_each_car

            list_of_car_features = soup2.select_one('html body div#page.main div div#spec div#pbox.detail-cover').find_all("div", {"class": "tr"})
            for each in list_of_car_features:
                if each.find_all("div")[0].text.strip() :
                
                    feature_name= each.find_all("div")[0].text.strip().lower().replace(" ", '_')
                    feature_data= each.find_all("div")[1].text.strip()
                    
                    ##
                    car_details_grouped_by_brand[feature_name] = feature_data

            if n == 0 and m == 0:
                inner_df = pd.DataFrame(car_details_grouped_by_brand, index=[index_number])
                index_number += 1
                all_car_details = inner_df.copy()
            else:
                inner_df = pd.DataFrame(car_details_grouped_by_brand, index=[index_number])
                index_number += 1
                all_car_details = pd.concat([all_car_details, inner_df], join='outer').copy()
            
            time.sleep(0.1)
            
            clear_output(wait=True)
            print(f"Number of scraped car data: {index_number+1}/{total_cars_details}")
            



Number of scraped car data: 4160/4160


In [41]:
all_car_details.shape

(4159, 102)

In [43]:
all_car_details

,brand,year,price_in_kes,price_in_usd,image_url,car_link,model_number,made_in,warranty,available_colors,...,dvd_player,radio,smart_access_card_entry,leather_seats,voice_control,removable/convertible_top,shock_absorbers_type,assembled_in,cassette_player,introduction_date
0,Honda,2025,4366650,33850,https://www.ccarprice.com/products/Honda_CR-V_...,https://www.ccarprice.com/ke/honda-cr-v-ex-awd...,CR-V EX AWD 2025,japan,"Basic 3 Years / 36,000 Miles Corrosion 5 Years...","Crystal Black Pearl, Lunar Silver Metallic, Mo...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Honda,2025,4173150,32350,https://www.ccarprice.com/products/Honda_CR-V_...,https://www.ccarprice.com/ke/honda-cr-v-ex-2wd...,CR-V EX 2WD 2025,japan,"Basic 3 Years / 36,000 Miles Corrosion 5 Years...","Crystal Black Pearl, Lunar Silver Metallic, Mo...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Honda,2025,2438100,18900,https://www.ccarprice.com/products/Honda_Eleva...,https://www.ccarprice.com/ke/honda-elevate-sig...,Elevate Signature Black Edition 2025,NaN,3 Years / Unlimited kilometers,crystal black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Honda,2025,2309100,17900,https://www.ccarprice.com/products/Honda_Eleva...,https://www.ccarprice.com/ke/honda-elevate-bla...,Elevate Black Edition 2025,NaN,3 Years / Unlimited kilometers,crystal black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Honda,2026,6372600,49400,https://www.ccarprice.com/products/Honda_Pilot...,https://www.ccarprice.com/ke/honda-pilot-trail...,2026 Honda Pilot TrailSport,japan,"3-year / 36,000 miles","Lunar Silver Metallic, Crystal Black Pearl, Mo...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4154,Xiaomi,2025,14731800,114200,https://www.ccarprice.com/products/2025_Xiaomi...,https://www.ccarprice.com/ke/xiaomi-su7-ultra-...,SU7 Ultra 2025,China,NaN,"Galaxy Gray, Yellow, Twilight Red, Mist Purple...",...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4155,Xiaomi,2025,3986100,30900,https://www.ccarprice.com/products/2025_Xiaomi...,https://www.ccarprice.com/ke/xiaomi-mx11-coupe...,MX11 Coupe SUV 2025,Beijing,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4156,Xiaomi,2025,3870000,30000,https://www.ccarprice.com/products/Xiaomi_SU8_...,https://www.ccarprice.com/ke/xiaomi-su8-midsiz...,SU8 Midsize SUV 2025,China,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4157,Xiaomi,2024,5269650,40850,https://www.ccarprice.com/products/Xiaomi_SU7_...,https://www.ccarprice.com/ke/xiaomi-su7-max-aw...,SU7 Max AWD 2024,Beijing,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Saving the initial dataframe
all_car_details.to_csv('Car_Prices_2.csv', index=False)